In [ ]:
%load_ext kamu
import kamu
con = kamu.connect()

In [ ]:
%%sql
select * from "world.data.tdreabing.nuclear-weapon-explosions" limit 3

In [ ]:
%%sql
select count(*) from "world.data.tdreabing.nuclear-weapon-explosions"

In [ ]:
%%sql
describe "world.data.tdreabing.nuclear-weapon-explosions"

In [ ]:
%%sql
select
    country,
    count(*) as count 
from "world.data.tdreabing.nuclear-weapon-explosions"
group by country

In [ ]:
%%sql -o df -q
select * from "world.data.tdreabing.nuclear-weapon-explosions"

In [ ]:
import os

import pandas as pd

from mapboxgl.utils import create_color_stops, df_to_geojson
from mapboxgl.viz import CircleViz, ClusteredCircleViz

# Must be a public token, starting with `pk`
token = os.getenv('MAPBOX_ACCESS_TOKEN')

# Create a geojson file export from a Pandas dataframe
df['date'] = df['event_time'].astype(str)
gj = df_to_geojson(
    df, properties=[
        'date', 'source', 'country', 'name', 'confirmation', 
        'medium', 'depth', 'mb', 'max_yield', 'salvo',
    ],
    lat='latitude', lon='longitude', precision=3)

# Generate data breaks and color stops from colorBrewer
color_breaks = [0,20,40,60,80,100]
color_stops = create_color_stops(color_breaks, colors='YlOrRd')

# Create the viz from the dataframe
viz = CircleViz(
    gj,
    style='mapbox://styles/mapbox/dark-v10',
    access_token=token,
    radius=5,
    stroke_color='black',
    stroke_width=0.2,
    height='600px',
    color_property="max_yield",
    color_stops=color_stops,
    center=(-95, 40),
    zoom=3,
    below_layer = 'waterway-label'
)
viz.show()

In [ ]:
viz = ClusteredCircleViz(
    gj,
    style='mapbox://styles/mapbox/dark-v10',
    height='600px',
    access_token=token,
    color_stops=color_stops,
    radius_stops=[[1,5], [10, 10], [50, 15], [100, 20]],
    radius_default=4,
    cluster_maxzoom=10,
    cluster_radius=15,
    label_size=12,
    opacity=0.9,
    center=(-95, 40),
    zoom=3,
)

viz.show()